In [1]:
#this notbook is for testing out little bits of code that could be useful in the overall pipeline. API calls, reading alignments, running other software stuff etc
#I've added some comments in cells for some small tasks we might need to do often. It would be good to have some sanity check outputs for each task to make sure the code is working


In [9]:
#setup folders
import os
import glob
folders = [ 'alns' , 'templates' , 'TensorflowModels' ]
clear = False

for path in folders:
    if not os.path.isdir(path):
        os.mkdir(path)
    if clear == True:
        files = glob.glob(path+'*.pdb')
        for f in files:
            os.remove(f)

In [2]:
#dl pdb files
#this uses wget to download individual pdb files and stores the paths
import wget
dl_url = 'http://files.rcsb.org/download/'
dl_url_err = 'http://files.rcsb.org/download/'
structs = {}
already = glob.glob( './templates/*.pdb' )
print(already)
#pull complexes
for m in models:
    structfile = './templates/'+m.upper().strip()+'.pdb'
    if structfile not in already:
        print(m)
        time.sleep(1)
        try:
            wget.download(url = dl_url + m.strip() +'.pdb' , out =structfile)
            structs[m] = structfile
        except:
            try:
                wget.download(url = dl_url + m.strip() +'.pdb' , out =structfile)
                structs[m] = structfile
            except:
                print('err', m )
    else:
        structs[m.strip()] = structfile
        
        

In [11]:
#grab sequences in AA from the PDB files 


from Bio import Alphabet
from Bio.SeqUtils import seq1
from Bio.PDB.PDBParser import PDBParser

parser = PDBParser()
#converter from 3 letter codes to 1 letter
letter3 = Alphabet.ThreeLetterProtein.letters
converter = { l.upper(): seq1(l) for l in letter3 }

structseqs={}
with open( 'structs.fast' , 'w')as fastout:
    for s in structs:
        Structure = PDBParser().get_structure(s, structs[s])
        for model in Structure:
            for chain in model:
                res = chain.get_residues()
                seq =  ''.join([ converter[r.get_resname()] for r in res if r.get_resname() in converter ] )
                fastout.write('>' + s + '|'+ chain.id +'\n')
                fastout.write(str( seq ) +'\n'  )
                structseqs[ s + '|'+ chain.id ] = seq

                
                

NameError: name 'structs' is not defined

In [ ]:


#proto for a function that opens an external program. pretty useful since we're using a lot of 3rd party tools

import subprocess 
import shlex

def runclustalo( aln , name, path = 'clsutalo' , outdir='./', verbose = False, SS = False):
    if verbose == True:
        print( [aln , name, path , outdir] )
    outhhm= outdir+name+".hhm"
    args = path + ' -i '+  aln  +' -o '+ outhhm + ' -M 50'
    if SS == True:
        #todo : make ss prediction here
        pass
    args = shlex.split(args)
    print(args)
    p = subprocess.Popen(args )
    return p , [outhhm]

    

In [ ]:
# add the amino acid sequence from the 3d structure to an aligment
# run clustalo merge



In [3]:
#read aln 2 numpy matrix
import numpy
from Bio import AlignIO

alnfile = ''
msa = AlignIO.read(alnfile , format = 'fasta')
align_array = np.array([ list(rec.upper())  for rec in msa], np.character)

In [4]:
#run dssp on a struct




In [ ]:
#run protFet on a sequence




In [6]:
#pdb 2 distmat




In [7]:
#fft ndim



#Ndimensional robust scaler


class NDSRobust(TransformerMixin):
    def __init__(self, **kwargs):
        self._scaler = RobustScaler(copy=True, **kwargs)
        self._orig_shape = None

    def fit(self, X, **kwargs):
        X = np.array(X)
        # Save the original shape to reshape the flattened X later
        # back to its original shape
        
        if len(X.shape) > 1:
            self._orig_shape = X.shape[1:]
        X = self._flatten(X)
        self._scaler.fit(X, **kwargs)
        return self

    def transform(self, X, **kwargs):
        X = np.array(X)
        X = self._flatten(X)
        X = self._scaler.transform(X, **kwargs)
        X = self._reshape(X)
        return X
    
    def inverse_transform(self, X, **kwargs):
        X = np.array(X)
        X = self._flatten(X)
        X = self._scaler.inverse_transform(X, **kwargs)
        X = self._reshape(X)
        return X
    
    def _flatten(self, X):
        # Reshape X to <= 2 dimensions
        if len(X.shape) > 2:
            n_dims = np.prod(self._orig_shape)
            X = X.reshape(-1, n_dims)
        return X

    def _reshape(self, X):
        # Reshape X back to it's original shape
        if len(X.shape) >= 2:
            X = X.reshape(-1, *self._orig_shape)
        return X


    
class NDSPCA(TransformerMixin):
    def __init__(self, **kwargs):
        self._scaler = PCA(copy = True, **kwargs)
        self._orig_shape = None

    def fit(self, X, **kwargs):
        X = np.array(X)
        # Save the original shape to reshape the flattened X later
        # back to its original shape
        
        if len(X.shape) > 1:
            self._orig_shape = X.shape[1:]
        X = self._flatten(X)
        self._scaler.fit(X, **kwargs)
        self.explained_variance_ratio_ = self._scaler.explained_variance_ratio_
        self.components_ =self._scaler.components_
        
        return self

    def transform(self, X, **kwargs):
        X = np.array(X)
        X = self._flatten(X)
        X = self._scaler.transform(X, **kwargs)
        
        return X
    
    def inverse_transform(self, X, **kwargs):
        X = np.array(X)
        X = self._flatten(X)
        X = self._scaler.inverse_transform(X, **kwargs)
        X = self._reshape(X)
        return X

    def _flatten(self, X):
        # Reshape X to <= 2 dimensions
        if len(X.shape) > 2:
            n_dims = np.prod(self._orig_shape)
            X = X.reshape(-1, n_dims)
        return X

    def _reshape(self, X):
        # Reshape X back to it's original shape
        if len(X.shape) >= 2:
            X = X.reshape(-1, *self._orig_shape)
        return X
    

def fit_y( y , components = 300 , FFT = False ):
    if FFT == True:
        y = np.stack([ np.fft.rfft2(y[i,:,:]) for i in range(y.shape[0])] )
        print(y.shape)
        y =  np.hstack( [ np.real(y) , np.imag(y)]  )
    print(y.shape)
    ndpca = NDSPCA(n_components=components)
    ndpca.fit(y)
    print('explained variance')
    print(np.sum(ndpca.explained_variance_ratio_))
    y = ndpca.transform(y)
    scaler0 = RobustScaler( )
    scaler0.fit(y)
    return scaler0, ndpca 

def transform_y( y, scaler,ndpca , FFT = False ):
    if FFT == True:
        y = np.stack([ np.fft.rfft2(y[i,:,:]) for i in range(y.shape[0])] )
        print(y.shape)
        y =  np.hstack( [ np.real(y) , np.imag(y)]  )
    y = ndpca.transform(y)
    print(y.shape)
    y = scaler0.transform(y)
    
    return y 

def inverse_transform_y( y , scaler, ndpca , FFT=False):
    
    y = scaler0.inverse_transform(y)
    y = ndpca.inverse_transform(y)

    if FFT == True:
        split = int(y.shape[1]/2)
        y = np.stack([ np.fft.irfft2(y[i,:split,:] + 1j*y[i,split:,:]) for i in range(y.shape[0]) ] )
                  
    return y
